In [18]:
import cv2 as cv

import numpy as np

import mediapipe as mp



In [19]:
import cv2
print(cv2)



<module 'cv2' from 'c:\\Users\\HP\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\cv2\\__init__.py'>


In [20]:
mp_pose  = mp.solutions.pose


mp_drawings = mp.solutions.drawing_utils

In [ ]:


def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    ##converting radians into angle
    angle = np.abs(radians * 180.0 / np.pi)
    ##normalizing the angle
    if angle > 180:
        angle = 360 - angle

    return angle


video = cv.VideoCapture('videoplayback.mp4')

max_jump_height = 0
takeoff_height = None

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:

    while video.isOpened():

        ret, frame = video.read()
        if not ret:
            break

        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = pose.process(image)
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        if result.pose_landmarks:

            landmarks = result.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            knee_angle = calculate_angle(hip, knee, ankle)

            current_height = ankle[1]

            if takeoff_height is None:
                takeoff_height = current_height

            jump_height = takeoff_height - current_height

            if jump_height > max_jump_height:
                max_jump_height = jump_height

            elbow_score = min(elbow_angle / 180, 1)
            knee_score = 1 - abs(knee_angle - 100) / 100
            jump_score = min(max_jump_height * 5, 1)

            smash_score = round(
                (elbow_score * 0.4 +
                 knee_score * 0.3 +
                 jump_score * 0.3) * 100, 2)

            cv.putText(image, f'Elbow Angle: {int(elbow_angle)}',(30, 50), cv.FONT_HERSHEY_SIMPLEX,0.8, (0, 255, 0), 2)

            cv.putText(image, f'Knee Angle: {int(knee_angle)}',(30, 90), cv.FONT_HERSHEY_SIMPLEX,0.8, (0, 255, 0), 2)

            cv.putText(image, f'Jump Height: {round(max_jump_height, 2)}',(30, 130), cv.FONT_HERSHEY_SIMPLEX,0.8, (255, 0, 0), 2)

            cv.putText(image, f'Smash Score: {smash_score}%',(30, 170), cv.FONT_HERSHEY_SIMPLEX,0.9, (0, 0, 255), 3)

        mp_drawings.draw_landmarks(
            image,
            result.pose_landmarks,
            mp_pose.POSE_CONNECTIONS)

        cv.imshow('Badminton performance analyzer', image)

        if cv.waitKey(10) & 0xFF == ord(' '):
            break

video.release()
cv.destroyAllWindows()